# Applying corrections to columnar data

Here we will show how to use the `coffea.lookup_tools` package.
It is able to read in a variety of common correction file formats into a standardized lookup table format.
We also cover here some CMS-specific tools for jet corrections (`coffea.jetmet_tools`) and b-tagging efficiencies/uncertainties (`coffea.btag_tools`).

**Test data**:
We'll use NanoEvents to construct some test data.

In [ ]:
from coffea.nanoaod import NanoEvents

fname = "https://github.com/CoffeaTeam/coffea/raw/master/tests/samples/nano_dy.root"
events = NanoEvents.from_file(fname)

The entrypoint for `coffea.lookup_tools` is the [extractor class](https://coffeateam.github.io/coffea/api/coffea.lookup_tools.extractor.html#coffea.lookup_tools.extractor).

In [ ]:
from coffea.lookup_tools import extractor

In [ ]:
%%bash
# download some sample correction sources
mkdir -p data
pushd data
PREFIX=https://raw.githubusercontent.com/CoffeaTeam/coffea/master/tests/samples
curl -Os $PREFIX/testSF2d.histo.root
curl -Os $PREFIX/testBTagSF.btag.csv
curl -Os $PREFIX/EIDISO_WH_out.histo.json
curl -Os $PREFIX/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt
curl -Os $PREFIX/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt
curl -Os $PREFIX/DeepCSV_102XSF_V1.btag.csv.gz
popd

## Opening a root file and using it as a lookup table

In [tests/samples](https://github.com/CoffeaTeam/coffea/tree/master/tests/samples), there is an example file with a `TH2F` histogram named `scalefactors_Tight_Electron`. The following code reads that histogram into an [evaluator](https://coffeateam.github.io/coffea/api/coffea.lookup_tools.evaluator.html#coffea.lookup_tools.evaluator) instance, under the key `testSF2d` and applies it to some electrons.

In [ ]:
ext = extractor()
# several histograms can be imported at once using wildcards (*)
ext.add_weight_sets(["testSF2d scalefactors_Tight_Electron data/testSF2d.histo.root"])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)
print("testSF2d:", evaluator['testSF2d'])
print("type of testSF2d:", type(evaluator['testSF2d']))

In [ ]:
print("Electron eta:", events.Electron.eta)
print("Electron pt:", events.Electron.pt)
print("Scale factor:", evaluator["testSF2d"](events.Electron.eta, events.Electron.pt))

## Reading a CMS b-tag scale factor csv file

These files have the following structure:

In [ ]:
%%bash
head -5 data/testBTagSF.btag.csv

The extractor assumes `*.csv` files have this structure and interprets them as so. The resulting scale factors can be used to calculate b-tagging corrections or uncertainties. **Note**: a high-level b-tagging correction class is also available, see the later sections of this guide.

In [ ]:
ext = extractor()
ext.add_weight_sets(["testBTag * data/testBTagSF.btag.csv"])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for i, key in enumerate(evaluator.keys()):
    print("\t", key)
    if i > 5:
        print("\t ...")
        break
print("testBTagCSVv2_1_comb_up_0:", evaluator['testBTagCSVv2_1_comb_up_0'])
print("type of testBTagCSVv2_1_comb_up_0:", type(evaluator['testBTagCSVv2_1_comb_up_0']))

In [ ]:
# note: in a real situation you would want to apply the SF on the appropriate jet flavor
scalefactor = evaluator['testBTagCSVv2_1_comb_up_0'](events.Jet.eta, events.Jet.pt, events.Jet.btagCSVV2)
print(scalefactor)

## Importing JSON-encoded histograms

Some corrections are provided in a json format, with a structure like
```
data[category][name][axis1 bin][axis2 bin]["value"]
```

In [ ]:
%%bash
head -10 data/EIDISO_WH_out.histo.json

The extractor assumes `*.json` files follow this format.

In [ ]:
ext = extractor()
ext.add_weight_sets(["* * data/EIDISO_WH_out.histo.json"])
ext.finalize()
    
evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)
print("EIDISO_WH/eta_pt_ratio_value:", evaluator['EIDISO_WH/eta_pt_ratio_value'])
print("type of EIDISO_WH/eta_pt_ratio_value:", type(evaluator['EIDISO_WH/eta_pt_ratio_value']))

In [ ]:
sf_out = evaluator['EIDISO_WH/eta_pt_ratio_value'](events.Electron.eta, events.Electron.pt)
sf_err_out = evaluator['EIDISO_WH/eta_pt_ratio_error'](events.Electron.eta, events.Electron.pt)
print(sf_out)
print(sf_err_out)

## Import CMS Jet Energy Scales and Uncertainties
In CMS, the jet energy scale and resolution corrections, as well as their uncertainties are available in a standalone text file format:

In [ ]:
%%bash
head -5 data/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt

In [ ]:
%%bash
head -3 data/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt

The extractor assumes files with `*.txt` are to be interpreted as such

In [ ]:
ext = extractor()
ext.add_weight_sets([
    "* * data/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt",
    "* * data/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt",
])
ext.finalize()

evaluator = ext.make_evaluator()

print("available evaluator keys:")
for key in evaluator.keys():
    print("\t", key)

print()
print("Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi:")
print(evaluator['Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi'])
print("type of Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi:")
print(type(evaluator['Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi']))
print()
print("Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi:")
print(evaluator['Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi'])
print("type of Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi:")
print(type(evaluator['Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi']))

In [ ]:
jec_out = evaluator['Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi'](events.Jet.eta, events.Jet.pt)
junc_out = evaluator['Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi'](events.Jet.eta, events.Jet.pt)
print("Correction factor:", jec_out)
# junc_out is a double-jagged array, with the last index being the up, down values
# they can be separated via indexing, e.g.:
print("Uncertainty +:", junc_out[:, :, 0])
print("Uncertainty -:", junc_out[:, :, 1])

## Applying energy scale transformations to Jets

The `coffea.jetmet_tools` package provides a convenience class [JetTransformer](https://coffeateam.github.io/coffea/api/coffea.jetmet_tools.JetTransformer.html#coffea.jetmet_tools.JetTransformer) which applies specified corrections and computes uncertainties in one call.

In [ ]:
from coffea.analysis_objects import JaggedCandidateArray
from coffea.jetmet_tools import FactorizedJetCorrector, JetCorrectionUncertainty
from coffea.jetmet_tools import JetTransformer

ext = extractor()
ext.add_weight_sets([
    "* * data/Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi.jec.txt",
    "* * data/Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi.junc.txt",
])
ext.finalize()

evaluator = ext.make_evaluator()

print(dir(evaluator))
print()

jets = JaggedCandidateArray.candidatesfromcounts(
    events.Jet.counts,
    pt=(events.Jet.pt * (1 - events.Jet.rawFactor)).flatten(),
    eta=events.Jet.y.flatten(),
    phi=events.Jet.z.flatten(),
    mass=(events.Jet.mass * (1 - events.Jet.rawFactor)).flatten(),
)
jets.add_attributes(ptRaw=jets.pt, massRaw=jets.mass)

corrector = FactorizedJetCorrector(
    Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi=evaluator['Fall17_17Nov2017_V32_MC_L2Relative_AK4PFPuppi'],
)
uncertainties = JetCorrectionUncertainty(
    Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi=evaluator['Fall17_17Nov2017_V32_MC_Uncertainty_AK4PFPuppi']
)

transformer = JetTransformer(jec=corrector, junc=uncertainties)
### more possibilities are available if you send in more pieces of the JEC stack
# mc2016_ak8_jxform = JetTransformer(jec=MC_AK8JEC2016,junc=MC_AK8JUNC2016
#                                    jer=MC_AK8JER2016,jersf=MC_AK8JERSF2016)

print()
print('starting columns:',jets.columns)
print()

print('untransformed pt ratios',jets.pt/jets.ptRaw)
print('untransformed mass ratios',jets.mass/jets.massRaw)

transformer.transform(jets)

print('transformed pt ratios',jets.pt/jets.ptRaw)
print('transformed mass ratios',jets.mass/jets.massRaw)

print()
print('transformed columns:',jets.columns)
print()

print('JES UP pt ratio',jets.pt_jes_up/jets.ptRaw)
print('JES DOWN pt ratio',jets.pt_jes_down/jets.ptRaw)

## Applying CMS b-tagging corrections
The `coffea.btag_tools` module provides the high-level utility [BTagScaleFactor](https://coffeateam.github.io/coffea/api/coffea.btag_tools.BTagScaleFactor.html#coffea.btag_tools.BTagScaleFactor) which calculates per-jet weights for b-tagging as well as light flavor mis-tagging efficiencies. Uncertainties can be calculated as well.

In [ ]:
from coffea.btag_tools import BTagScaleFactor

btag_sf = BTagScaleFactor("data/DeepCSV_102XSF_V1.btag.csv.gz", "medium")

print("SF:", btag_sf.eval("central", events.Jet.hadronFlavour, abs(events.Jet.eta), events.Jet.pt))
print("systematic +:", btag_sf.eval("up", events.Jet.hadronFlavour, abs(events.Jet.eta), events.Jet.pt))